<div class="alert alert-block alert-info">
<b>Deadline:</b> March 30, 2022 (Wednesday) 23:00
</div>

# Exercise 1. Sequence-to-sequence modeling with recurrent neural networks

The goals of this exercise are
* to get familiar with recurrent neural networks used for sequential data processing
* to get familiar with the sequence-to-sequence model for machine translation
* to learn PyTorch tools for batch processing of sequences with varying lengths
* to learn how to write a custom `DataLoader`

You may find it useful to look at this tutorial:
* [Translation with a Sequence to Sequence Network and Attention](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)

In [1]:
skip_training = False  # Set this flag to True before validation and submission

In [2]:
# During evaluation, this cell sets skip_training to True
# skip_training = True

import tools, warnings
warnings.showwarning = tools.customwarn

In [3]:
import os
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader

import tools
import tests

In [4]:
# When running on your own computer, you can specify the data directory by:
# data_dir = tools.select_data_dir('/your/local/data/directory')
data_dir = tools.select_data_dir()

The data directory is /coursedata


In [5]:
# Select the device for training (use GPU if you have one)
#device = torch.device('cuda:0')
device = torch.device('cpu')

In [6]:
if skip_training:
    # The models are always evaluated on CPU
    device = torch.device("cpu")

## Data

The dataset that we are going to use consists of pairs of sentences in French and English.

In [7]:
from data import TranslationDataset, MAX_LENGTH, SOS_token, EOS_token

trainset = TranslationDataset(data_dir, train=True)

* `TranslationDataset` supports indexing as required by `torch.utils.data.Dataset`.
* Sentences are tensors of maximum length `MAX_LENGTH`.
* Words in a (sentence) tensor are represented as an index (integer) in a language vocabulary.
* The string representation of a word from the source language can be obtained from index `i` with `dataset.input_lang.index2word[i]`.
* Similarly for the target language `dataset.output_lang.index2word[j]`.

Let us look at samples from that dataset.

In [8]:
src_sentence, tgt_sentence = trainset[np.random.choice(len(trainset))]
print('Source sentence: "%s"' % ' '.join(trainset.input_lang.index2word[i.item()] for i in src_sentence))
print('Sentence as tensor of word indices:')
print(src_sentence)

print('Target sentence: "%s"' % ' '.join(trainset.output_lang.index2word[i.item()] for i in tgt_sentence))
print('Sentence as tensor of word indices:')
print(tgt_sentence)

Source sentence: "vous etes rusees . EOS"
Sentence as tensor of word indices:
tensor([118, 215, 853,   5,   1])
Target sentence: "you re crafty . EOS"
Sentence as tensor of word indices:
tensor([130,  78, 454,   4,   1])


In [9]:
print('Number of source-target pairs in the training set: ', len(trainset))

Number of source-target pairs in the training set:  8682


## Sequence-to-sequence model for machine translation

In this exercise, we are going to build a machine translation system which transforms a sentence in one language into a sentence in another one. The computational graph of the translation model is shown below:

<img src="seq2seq.png" width=900>

We are going to use a simplified model without the dotted connections.

## Custom DataLoader

We would like to train the sequence-to-sequence model using mini-batch training.
One difficulty of mini-batch training in this case is that sequences may have varying lengths and this has to be taken into account when building the computational graph. Luckily, PyTorch has tools to support batch processing of such sequences.
To use those tools, we need to write a custom data loader which puts sequences of varying lengths in the same tensor. We can customize the data loader by providing a custom `collate_fn` as explained [here](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader).

Our collate function:
- combines sequences from the source language in a single tensor with extra values (at the end) filled with `PADDING_VALUE=0`.
- combines sequences from the target language in a single tensor with extra values (at the end) filled with `PADDING_VALUE=0`.

**Important**:
- Later in the code (not in this `collate` function), we will convert source sequences to objects of class [`PackedSequence`](https://pytorch.org/docs/stable/nn.html?highlight=packedsequence#torch.nn.utils.rnn.PackedSequence) which can be processed by recurrent units such as `GRU` or `LSTM`. Conversion to `PackedSequence` objects requires sequences to be sorted by their lengths.
**Therefore, the returned source sequences should be sorted by length in a decreasing order.**
* The target sequences need not be sorted by their lengths because we have to keep the same order of sequences in the source and target tensors.

Your task is to implement the collate function.

In [10]:
PADDING_VALUE = 0

In [11]:
#my code  test
from torch.nn.utils.rnn import pad_sequence
pairs = [
        (torch.LongTensor([1, 2]), torch.LongTensor([3, 4, 5])),
        (torch.LongTensor([6, 7, 8]), torch.LongTensor([9, 10])),
    ]
print(pairs[0][0])
test=torch.cat((pairs[0][0], pairs[0][1]),dim=0)
print(test)
#print(test.reshape(3,2))
#len(test)
len(pairs)
print(pairs[0][0])
a=pairs[0][0].clone()
b=pairs[0][1].clone()

print(a,b)
src_seqs = [sample[0] for sample in pairs]
print("test for function",src_seqs)
src_seqs = pad_sequence(src_seqs)
print("test for pad",src_seqs)

tensor([1, 2])
tensor([1, 2, 3, 4, 5])
tensor([1, 2])
tensor([1, 2]) tensor([3, 4, 5])
test for function [tensor([1, 2]), tensor([6, 7, 8])]
test for pad tensor([[1, 6],
        [2, 7],
        [0, 8]])


In [12]:
from torch.nn.utils.rnn import pad_sequence

def collate(list_of_samples):
    """Merges a list of samples to form a mini-batch.

    Args:
      list_of_samples is a list of tuples (src_seq, tgt_seq):
          src_seq is of shape (src_seq_length,)
          tgt_seq is of shape (tgt_seq_length,)

    Returns:
      src_seqs of shape (max_src_seq_length, batch_size): Tensor of padded source sequences.
          The sequences should be sorted by length in a decreasing order, that is src_seqs[:,0] should be
          the longest sequence, and src_seqs[:,-1] should be the shortest.
      src_seq_lengths: List of lengths of source sequences.
      tgt_seqs of shape (max_tgt_seq_length, batch_size): Tensor of padded target sequences.
    """
    # YOUR CODE HERE
    
    #padnp=np.array([PADDING_VALUE])
    #padding= torch.LongTensor(padnp)
    
    #temp=torch.cat((list_of_samples[0][0], padding),dim=0)
    #print(temp)
    #src_seqs=torch.stack([list_of_samples[1][0],temp], dim=1)
    
    #print("sorted:  ",src_seqs)
  
    #print(src_seq_lengths,type(src_seq_lengths))
        
    #temp=torch.cat((list_of_samples[1][1], padding),dim=0)
    #print(temp)
    #tgt_seqs=torch.stack((temp,list_of_samples[0][1]),dim=1)
    
    #print(tgt_seqs)
    
    
    
    
    src_seqs = list_of_samples[0]
    print("src_origin",src_seqs)
    
    src_seqs = sorted(list_of_samples[0],key=lambda x: len(x) ,reverse=True)
    print("src_sorted",src_seqs)
    
    src_seq_lengths =[len(i) for i in src_seqs]
    src_seqs = pad_sequence(src_seqs)
    print("src_pad",src_seqs)
    tgt_seqs = list_of_samples[1]
    print("tgt_origin",tgt_seqs)
    tgt_seqs = pad_sequence(tgt_seqs)
    print("tgt_pad",tgt_seqs)
    
    return src_seqs, src_seq_lengths, tgt_seqs
    raise NotImplementedError()

In [13]:
def test_collate_shapes():
    pairs = [
        (torch.LongTensor([1, 2]), torch.LongTensor([3, 4, 5])),
        (torch.LongTensor([6, 7, 8]), torch.LongTensor([9, 10])),
    ]
    pad_src_seqs, src_seq_lengths, pad_tgt_seqs = collate(pairs)
    assert type(src_seq_lengths) == list, "src_seq_lengths should be a list."
    assert pad_src_seqs.shape == torch.Size([3, 2]), f"Bad pad_src_seqs.shape: {pad_src_seqs.shape}"
    assert pad_src_seqs.dtype == torch.long
    assert pad_tgt_seqs.shape == torch.Size([3, 2]), f"Bad pad_tgt_seqs.shape: {pad_tgt_seqs.shape}"
    assert pad_tgt_seqs.dtype == torch.long
    print('Success')

test_collate_shapes()

src_origin (tensor([1, 2]), tensor([3, 4, 5]))
src_sorted [tensor([3, 4, 5]), tensor([1, 2])]
src_pad tensor([[3, 1],
        [4, 2],
        [5, 0]])
tgt_origin (tensor([6, 7, 8]), tensor([ 9, 10]))
tgt_pad tensor([[ 6,  9],
        [ 7, 10],
        [ 8,  0]])
Success


In [14]:
list indices must be integers or slices, not tuple# This cell tests collate() function

SyntaxError: invalid syntax (3194508349.py, line 1)

In [15]:
# We create custom DataLoader using the implemented collate function
# We are going to process 64 sequences at the same time (batch_size=64)
trainloader = DataLoader(dataset=trainset, batch_size=64, shuffle=True, collate_fn=collate, pin_memory=True)

## Encoder

The encoder encodes a source sequence $(x_1, x_2, ..., x_T)$ into a single vector $h_T$ using the following recursion:
$$
  h_{t} = f(h_{t-1}, x_t) \qquad t = 1, \ldots, T
$$
where:
* intial state $h_0$ is often chosen arbitrarily (we choose it to be zero)
* function $f$ is defined by the type of the RNN cell (in our experiments, we will use [GRU](https://pytorch.org/docs/stable/nn.html#torch.nn.GRU))
* $x_t$ is a vector that represents the $t$-th word in the source sentence.

A common practice in natural language processing is to _learn_ the word representations $x_t$ (instead of, for example, using one-hot coded vectors). In PyTorch, this is supported by class [Embedding](https://pytorch.org/docs/stable/nn.html#torch.nn.Embedding) which we are going to use.

The computational graph of the encoder is shown below:

<img src="seq2seq_encoder.png" width=500>

Your task is to implement the `forward` function of the encoder. It should contain the following steps:
* Embed the words of the source sequences.
* Pack source sequences using [`pack_padded_sequence`](https://pytorch.org/docs/stable/nn.html?highlight=pack_padded_sequence#torch.nn.utils.rnn.pack_padded_sequence). This converts padded source sequences into an object that can be processed by PyTorch recurrent units such as `nn.GRU` or `nn.LSTM`.
* Apply GRU computations to packed sequences obtained in the previous step
* Convert packed sequence of GRU outputs into padded representation with [`pad_packed_sequence`](https://pytorch.org/docs/stable/nn.html?highlight=pad_packed_sequence#torch.nn.utils.rnn.pad_packed_sequence).

In [16]:
class Encoder(nn.Module):
    def __init__(self, src_dictionary_size, embed_size, hidden_size):
        """
        Args:
          src_dictionary_size: The number of words in the source dictionary.
          embed_size: The number of dimensions in the word embeddings.
          hidden_size: The number of features in the hidden state of GRU.
        """
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(src_dictionary_size, embed_size)
        self.gru = nn.GRU(input_size=embed_size, hidden_size=hidden_size)

    def forward(self, pad_seqs, seq_lengths, hidden):
        """
        Args:
          pad_seqs of shape (max_seq_length, batch_size): Padded source sequences.
          seq_lengths: List of sequence lengths.
          hidden of shape (1, batch_size, hidden_size): Initial states of the GRU.

        Returns:
          outputs of shape (max_seq_length, batch_size, hidden_size): Padded outputs of GRU at every step.
          hidden of shape (1, batch_size, hidden_size): Updated states of the GRU.
        """
        # YOUR CODE HERE

        embedded = self.embedding(pad_seqs) 
        seq_pack=pack_padded_sequence(embedded,seq_lengths)
        print("seq_pack",seq_pack)
        output, hidden = self.gru(seq_pack, hidden)
        print("output1",output)
        output= pad_packed_sequence(output)[0]
        print("output2",output)
        
        return output, hidden
        raise NotImplementedError()

    def init_hidden(self, batch_size=1):
        return torch.zeros(1, batch_size, self.hidden_size)

In [17]:
def test_Encoder_shapes():
    hidden_size = 3
    encoder = Encoder(src_dictionary_size=5, embed_size=10, hidden_size=hidden_size)

    max_seq_length = 4
    batch_size = 2
    hidden = encoder.init_hidden(batch_size=batch_size)
    pad_seqs = torch.tensor([
        [        1,             2],
        [        2,     EOS_token],
        [        3, PADDING_VALUE],
        [EOS_token, PADDING_VALUE]
    ])

    outputs, new_hidden = encoder.forward(pad_seqs=pad_seqs, seq_lengths=[4, 2], hidden=hidden)
    assert outputs.shape == torch.Size([4, batch_size, hidden_size]), f"Bad outputs.shape: {outputs.shape}"
    assert new_hidden.shape == torch.Size([1, batch_size, hidden_size]), f"Bad new_hidden.shape: {new_hidden.shape}"
    print('Success')

test_Encoder_shapes()

seq_pack PackedSequence(data=tensor([[ 0.2255, -0.3653,  0.1422,  0.6880, -0.0565, -0.4408,  1.2305,  0.0942,
         -0.3216,  0.0971],
        [-0.6234,  0.1280,  0.7172, -1.9170,  1.5535,  1.8419,  1.2268, -0.3613,
         -1.0013,  0.0054],
        [-0.6234,  0.1280,  0.7172, -1.9170,  1.5535,  1.8419,  1.2268, -0.3613,
         -1.0013,  0.0054],
        [ 0.2255, -0.3653,  0.1422,  0.6880, -0.0565, -0.4408,  1.2305,  0.0942,
         -0.3216,  0.0971],
        [ 0.8690,  0.3577,  0.5667,  0.7924,  0.7158,  0.1322, -1.3069, -1.1617,
         -0.2814,  1.1468],
        [ 0.2255, -0.3653,  0.1422,  0.6880, -0.0565, -0.4408,  1.2305,  0.0942,
         -0.3216,  0.0971]], grad_fn=<PackPaddedSequenceBackward0>), batch_sizes=tensor([2, 2, 1, 1]), sorted_indices=None, unsorted_indices=None)
output1 PackedSequence(data=tensor([[-0.0171, -0.4218, -0.2154],
        [ 0.2970, -0.3088, -0.3348],
        [ 0.2050, -0.3036, -0.5231],
        [ 0.1211, -0.5981, -0.2194],
        [ 0.0570, -0.4

In [18]:
# This cell tests Encoder

## Decoder

The decoder takes as input the representation computed by the encoder and transforms it into a sentence in the target language. The computational graph of the decoder is shown below:

<img src="seq2seq_decoder.png" width=500 align="top">

* $z_0$ is the output of the encoder, that is $z_0 = h_5$, thus `hidden_size` of the decoder should be the same as `hidden_size` of the encoder.
* $y_{i}$ are the log-probabilities of the words in the target language, the dimensionality of $y_{i}$ is the size of the target dictionary.
* $z_{i}$ is mapped to $y_{i}$ using a linear layer `self.out` followed by `F.log_softmax` (because we use `nn.NLLLoss` loss for training).
* Each cell of the decoder is a GRU, it receives as inputs the previous state $z_{i-1}$ and relu of the **embedding** of the previous word. Thus, you need to embed the words of the target language as well. The previous word is taken as the word with the maximum log-probability.

Note that the decoder outputs a word at every step and the same word is used as the input to the recurrent unit at the next step. At the beginning of decoding, the previous word input is fed with a special word SOS which stands for "start of a sentence". During training, we know the target sentence for decoding, therefore we can feed the correct words $y_i$ as inputs to the recurrent unit.

There is one extra thing that it is wise to take care of. When the target sentence is fed to the decoder during training, the decoder learns to generate only the next word (this scenario is called "teacher forcing"). At test time, the decoder works differently: it generates the whole sequence using its own predictions as inputs at each step. Therefore, it makes sense to train the decoder to produce full sentences. In order to do that, we will alternate between two modes during training:
* "teacher forcing": the decoder is fed with the words in the target sequence
* no "teacher forcing": the decoder generates the output sequence using its own predictions. In this case, we will generate sequences of the same length as the length of the longest sequence in `pad_tgt_seqs` (if `pad_tgt_seqs` is not `None`) or of length `MAX_LENGTH` (if `pad_tgt_seqs` is `None`).

You need to implement the decoder which has the structure shown in the figure above.

Notes:
* `SOS_token` is imported at the beginning of the notebook.
* **Running this code on GPU sometimes fails producing a CUDA error (if you know the reason, please let us know).** If this happens to you, please train the model on CPU.

In [19]:
class Decoder(nn.Module):
    def __init__(self, tgt_dictionary_size, embed_size, hidden_size):
        """
        Args:
          tgt_dictionary_size: The number of words in the target dictionary.
          embed_size: The number of dimensions in the word embeddings.
          hidden_size: The number of features in the hidden state.
        """
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(tgt_dictionary_size, embed_size)
        self.gru = nn.GRU(input_size=embed_size, hidden_size=hidden_size)
        self.out = nn.Linear(hidden_size, tgt_dictionary_size)

    def forward(self, hidden, pad_tgt_seqs=None, teacher_forcing=False):
        """
        Args:
          hidden of shape (1, batch_size, hidden_size): States of the GRU.
          pad_tgt_seqs of shape (max_out_seq_length, batch_size): Tensor of words (word indices) of the
              target sentence. If None, the output sequence is generated by feeding the decoder's outputs
              (teacher_forcing has to be False).
          teacher_forcing (bool): Whether to use teacher forcing or not.

        Returns:
          outputs of shape (max_out_seq_length, batch_size, tgt_dictionary_size): Tensor of log-probabilities
              of words in the target language.
          hidden of shape (1, batch_size, hidden_size): New states of the GRU.

        Note: Do not forget to transfer tensors that you may want to create in this function to the device
        specified by `hidden.device`.
        """
        if pad_tgt_seqs is None:
            assert not teacher_forcing, 'Cannot use teacher forcing without a target sequence.'

        # YOUR CODE HERE

        print("hidden , pad_tgt_seqs : \n", hidden,pad_tgt_seqs)
        
        sos=torch.LongTensor(SOS_token*np.ones((1,len(hidden[0]))))
        print("sos : ", sos)
        print ("Teacher forcing： ",teacher_forcing )
        
        if teacher_forcing == True:   
            print("pad_tgt_seqs columns should be 2",pad_tgt_seqs)
            input=torch.cat((sos,pad_tgt_seqs[:-1][:]),dim=0)
            print("input: ",input)
            output = self.embedding(input)
            output = F.relu(output)
            output, hidden = self.gru(output, hidden)
            
            output = self.out(output)
            output = F.log_softmax(output, dim = 2)
        
        if teacher_forcing == False:
            input = sos
            
            if  pad_tgt_seqs == None:
                pad_tgt_len = MAX_LENGTH
            else:
                pad_tgt_len = len(pad_tgt_seqs)
            
            
            for i in range(pad_tgt_len):
                print("range : ",i)
                y = self.embedding(input)
                y = F.relu(y)
                y, hidden = self.gru(y, hidden)
                y = self.out(y)
                y = F.log_softmax(y, dim=2)
                print ("output y:" ,y)
                
                sentence=y
                
                # connect the words to form a sentence
                if i>=1:
                    sentence=torch.cat((sentence,y),dim=0)
                    output=sentence
                
                # new input is previous output y
                input=torch.argmax(y,dim=2)
                
                
        output.to(hidden.device)
        hidden.to(hidden.device)
        
        return output,  hidden       
        raise NotImplementedError()

In [20]:
def test_Decoder_shapes():
    hidden_size = 2
    tgt_dictionary_size = 5
    test_decoder = Decoder(tgt_dictionary_size, embed_size=10, hidden_size=hidden_size)

    max_seq_length = 4
    batch_size = 2
    pad_tgt_seqs = torch.tensor([
        [        1,             2],
        [        2,     EOS_token],
        [        3, PADDING_VALUE],
        [EOS_token, PADDING_VALUE]
    ])  # [max_seq_length, batch_size]

    hidden = torch.zeros(1, batch_size, hidden_size)
    outputs, new_hidden = test_decoder.forward(hidden, pad_tgt_seqs, teacher_forcing=False)

    assert outputs.size(0) <= 4, f"Too long output sequence: outputs.size(0)={outputs.size(0)}"
    assert outputs.shape[1:] == torch.Size([batch_size, tgt_dictionary_size]), \
        f"Bad outputs.shape[1:]={outputs.shape[1:]}"
    assert new_hidden.shape == torch.Size([1, batch_size, hidden_size]), f"Bad new_hidden.shape={new_hidden.shape}"

    outputs, new_hidden = test_decoder.forward(hidden, pad_tgt_seqs, teacher_forcing=True)
    assert outputs.shape == torch.Size([4, batch_size, tgt_dictionary_size]), \
        f"Bad shape outputs.shape={outputs.shape}"
    assert new_hidden.shape == torch.Size([1, batch_size, hidden_size]), f"Bad new_hidden.shape={new_hidden.shape}"

    # Generation mode
    outputs, new_hidden = test_decoder.forward(hidden, None, teacher_forcing=False)
    assert outputs.shape[1:] == torch.Size([batch_size, tgt_dictionary_size]), \
        f"Bad outputs.shape[1:]={outputs.shape[1:]}"
    assert new_hidden.shape == torch.Size([1, batch_size, hidden_size]), f"Bad new_hidden.shape={new_hidden.shape}"

    print('Success')

test_Decoder_shapes()

hidden , pad_tgt_seqs : 
 tensor([[[0., 0.],
         [0., 0.]]]) tensor([[1, 2],
        [2, 1],
        [3, 0],
        [1, 0]])
sos :  tensor([[0, 0]])
Teacher forcing：  False
range :  0
output y: tensor([[[-1.7608, -1.4521, -2.0596, -1.1360, -1.9283],
         [-1.7608, -1.4521, -2.0596, -1.1360, -1.9283]]],
       grad_fn=<LogSoftmaxBackward0>)
range :  1
output y: tensor([[[-1.5135, -1.7559, -1.5784, -1.6771, -1.5424],
         [-1.5135, -1.7559, -1.5784, -1.6771, -1.5424]]],
       grad_fn=<LogSoftmaxBackward0>)
range :  2
output y: tensor([[[-1.6809, -1.4481, -1.9183, -1.2493, -1.9294],
         [-1.6809, -1.4481, -1.9183, -1.2493, -1.9294]]],
       grad_fn=<LogSoftmaxBackward0>)
range :  3
output y: tensor([[[-1.4991, -1.7029, -1.5553, -1.6805, -1.6241],
         [-1.4991, -1.7029, -1.5553, -1.6805, -1.6241]]],
       grad_fn=<LogSoftmaxBackward0>)
hidden , pad_tgt_seqs : 
 tensor([[[0., 0.],
         [0., 0.]]]) tensor([[1, 2],
        [2, 1],
        [3, 0],
        [1, 0]]

In [21]:
# This cell tests Decoder

In [22]:
# This cell tests Decoder

In [23]:
# This cell tests Decoder

## Training of sequence-to-sequence model using mini-batches

Now we are going to train the sequence-to-sequence model on the toy translation dataset.

In [24]:
# Create the seq2seq model
hidden_size = embed_size = 256
encoder = Encoder(trainset.input_lang.n_words, embed_size, hidden_size).to(device)
decoder = Decoder(trainset.output_lang.n_words, embed_size, hidden_size).to(device)

In [25]:
teacher_forcing_ratio = 0.5

Implement the training loop in the cell below. In the training loop, we first encode source sequences using the encoder, then we decode the encoded state using the decoder. The decoder outputs log-probabilities of words in the target language. We need to use these log-probabilities and the indexes of the words in the target sequences to compute the loss.

The loss is
\begin{align*}
L = - \frac{1}{N} \sum_{n} \sum_{t=1}^{T_n}
\log p\left(\mathbf{y}_t^{(n)} \:\Bigl|\: \mathbf{y}_{<t}^{(n)}, \mathbf{X}^{(n)} \right)
\end{align*}
where $T_n$ is the length of the $n$-th target sequence and $N= \sum_{n=1} T_n$ is the total number of words in all the sentences of the mini-batch.

Recommended hyperparameters:
- Encoder optimizer: Adam with learning rate 0.001
- Decoder optimizer: Adam with learning rate 0.001
- Number of epochs: 30
- Toggle `teacher_forcing` on and off (for each mini-batch) according to the `teacher_forcing_ratio` specified above.

Hints:
- Training should proceed relatively fast.
- If you do well, the training loss should reach 0.1 in 30 epochs.
- Slight overlearning may happen (you can see that if you track the test error during training) but you can ignore this problem. 
- **Important:** When computing the loss, you need to ignore the padded values. This can easily be done by using argument `ignore_index` of function [`nll_loss`](
https://pytorch.org/docs/stable/nn.functional.html#torch.nn.functional.nll_loss).

In [35]:
if not skip_training:
    # YOUR CODE HERE
    Encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr = 0.001)
    Decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr = 0.001)
    n_epochs = 30
    criterion = F.nll_loss
    
    for i in range (n_epochs):
        for src_seqs, src_seq_lengths, tgt_seqs in trainloader:
            Encoder_optimizer.zero_grad()
            Decoder_optimizer.zero_grad()
            print("src_seq_lengths : ", src_seq_lengths)
            
            hidden = encoder.init_hidden(len(src_seq_lengths))
            output , hidden = encoder.forward(src_seqs, src_seq_lengths, hidden)
            print("encoder output_shape, hidden_shape", output.shape, hidden.shape)
            
            teacher_forcing = True if random.random() < teacher_forcing_ratio else False
            #teacher_forcing = True
            
            output, hidden = decoder.forward(hidden,tgt_sentence,teacher_forcing)
            print("output, tgt_seqs: \n\n", output.shape, tgt_seqs)
            
            output = torch.flatten(output,end_dim=1)
            print("output: ",output.shape)
            loss = criterion(output, tgt_seqs.view(-1), ignore_index = PADDING_VALUE)
            
            loss.backward()
            Encoder_optimizer.step()
            Decoder_optimizer.step()
    
    #raise NotImplementedError()

src_origin (tensor([ 118,  215,  974, 2066,    5,    1]), tensor([130,  78, 532, 428,   4,   1]))
src_sorted [tensor([ 118,  215,  974, 2066,    5,    1]), tensor([130,  78, 532, 428,   4,   1])]
src_pad tensor([[ 118,  130],
        [ 215,   78],
        [ 974,  532],
        [2066,  428],
        [   5,    4],
        [   1,    1]])
tgt_origin (tensor([ 349,  350, 3652,  115,  332,   96, 1792,    5,    1]), tensor([ 222,  125, 1343,  539, 1555,  616,  295,  988,    4,    1]))
tgt_pad tensor([[ 349,  222],
        [ 350,  125],
        [3652, 1343],
        [ 115,  539],
        [ 332, 1555],
        [  96,  616],
        [1792,  295],
        [   5,  988],
        [   1,    4],
        [   0,    1]])
src_seq_lengths :  [6, 6]
seq_pack PackedSequence(data=tensor([[ 0.9295, -0.5833,  1.0380,  ..., -0.2575,  0.9747, -1.2493],
        [ 0.5999, -0.4198,  1.5599,  ...,  0.2124,  0.3146, -0.8375],
        [ 0.5644, -1.1567, -0.4569,  ..., -0.4618, -0.1561,  0.8662],
        ...,
        [-

ValueError: Expected input batch_size (4) to match target batch_size (20).

In [34]:
# Save the model to disk (the pth-files will be submitted automatically together with your notebook)
# Set confirm=False if you do not want to be asked for confirmation before saving.
if not skip_training:
    tools.save_model(encoder, '1_rnn_encoder.pth', confirm=True)
    tools.save_model(decoder, '1_rnn_decoder.pth', confirm=True)

Do you want to save the model (type yes to confirm)? no
Model not saved.
Do you want to save the model (type yes to confirm)? no
Model not saved.


In [ ]:
if skip_training:
    hidden_size = 256
    encoder = Encoder(trainset.input_lang.n_words, embed_size, hidden_size)
    tools.load_model(encoder, '1_rnn_encoder.pth', device)
    
    decoder = Decoder(trainset.output_lang.n_words, embed_size, hidden_size)
    tools.load_model(decoder, '1_rnn_decoder.pth', device)

In [ ]:
# This cell tests training accuracy

## Evaluation

Next we need to implement a function that converts input sequences to output sequences using the trained sequence-to-sequence model.

Notes:
* Since we do not need to compute the gradients in the evaluation phase, we can speed up the computations by using the statement `with torch.no_grad():`.
* Please transfer the tensors to `device` inside this function.

In [ ]:
def translate(encoder, decoder, pad_src_seqs, src_seq_lengths):
    """Translate sequences from the source language to the target language using the trained model.
    
    Args:
      encoder (Encoder): Trained encoder.
      decoder (Decoder): Trained decoder.
      pad_src_seqs of shape (max_src_seq_length, batch_size): Padded source sequences.
      src_seq_lengths: List of source sequence lengths.
    
    Returns:
      out_seqs of shape (MAX_LENGTH, batch_size): LongTensor of word indices of the output sequences.
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
def test_translate_shapes():
    pad_src_seqs = torch.tensor([
        [1, 2],
        [2, 3],
        [3, 0],
        [4, 0]
    ])

    out_seqs = translate(encoder, decoder, pad_src_seqs, src_seq_lengths=[4, 2])
    assert out_seqs.shape == torch.Size([MAX_LENGTH, 2]), f"Wrong out_seqs.shape: {out_seqs.shape}"
    print('Success')

test_translate_shapes()

Let us now translate a few sentences from the training set and print the source, target, and produced output.

If you trained the model well enough, the model should memorize the training data well.

In [ ]:
def seq_to_tokens(seq, lang):
    'Convert a sequence of word indices into a list of words (strings).'
    sentence = []
    for i in seq:
        if i == EOS_token:
            break
        sentence.append(lang.index2word[i.item()])
    return(sentence)

def seq_to_string(seq, lang):
    'Convert a sequence of word indices into a sentence string.'
    return(' '.join(seq_to_tokens(seq, lang)))

In [ ]:
# Translate a few sentences from the training set
print('Translate training data:')
print('-----------------------------')
pad_src_seqs, src_seq_lengths, pad_tgt_seqs = next(iter(trainloader))
out_seqs = translate(encoder, decoder, pad_src_seqs, src_seq_lengths)

for i in range(5):
    print('SRC:', seq_to_string(pad_src_seqs[:,i], trainset.input_lang))
    print('TGT:', seq_to_string(pad_tgt_seqs[:,i], trainset.output_lang))
    print('OUT:', seq_to_string(out_seqs[:,i], trainset.output_lang))
    print('')

Now we translate random sentences from the test set. A well-trained model should output sentences that look similar to the target ones. The mistakes are usually done for words that were rare in the training set.

In [ ]:
testset = TranslationDataset(data_dir, train=False)
testloader = DataLoader(dataset=testset, batch_size=64, shuffle=False, collate_fn=collate)

In [ ]:
print('Translate test data:')
print('-----------------------------')
pad_src_seqs, src_seq_lengths, pad_tgt_seqs = next(iter(testloader))
out_seqs = translate(encoder, decoder, pad_src_seqs, src_seq_lengths)

for i in range(5):
    print('SRC:', seq_to_string(pad_src_seqs[:,i], testset.input_lang))
    print('TGT:', seq_to_string(pad_tgt_seqs[:,i], testset.output_lang))
    print('OUT:', seq_to_string(out_seqs[:,i], testset.output_lang))
    print('')

## Compute BLEU score

Let us now compute the [BLEU score](https://en.wikipedia.org/wiki/BLEU) for the translations produced by our model. We can use the PyTorch function [bleu_score](https://pytorch.org/text/data_metrics.html#torchtext.data.metrics.bleu_score) for that.

* **Your model should achieve a minimum BLEU score of 90 on the training set.**
* The BLEU score on the test set should be greater than 40.

The model can severly overfit to the training set and we do not cope with the overfitting problem in this exercise.

In [ ]:
from torchtext.data.metrics import bleu_score

In [ ]:
# Create translations for the training set
candidate_corpus = []
references_corpus = []
for pad_src_seqs, src_seq_lengths, pad_tgt_seqs in trainloader:
    out_seqs = translate(encoder, decoder, pad_src_seqs, src_seq_lengths)
    candidate_corpus.extend([seq_to_tokens(seq, trainset.output_lang) for seq in out_seqs.T])
    references_corpus.extend([[seq_to_tokens(seq, trainset.output_lang)] for seq in pad_tgt_seqs.T])

# Compute BLEU for translations
score = bleu_score(candidate_corpus, references_corpus)
print(f'BLEU score on training data: {score*100}')
assert score*100 > 90, "The BLEU score is too low."

In [ ]:
# Create translations for the test set
candidate_corpus = []
references_corpus = []
for pad_src_seqs, src_seq_lengths, pad_tgt_seqs in testloader:
    out_seqs = translate(encoder, decoder, pad_src_seqs, src_seq_lengths)
    candidate_corpus.extend([seq_to_tokens(seq, testset.output_lang) for seq in out_seqs.T])
    references_corpus.extend([[seq_to_tokens(seq, testset.output_lang)] for seq in pad_tgt_seqs.T])

# Compute BLEU for translations
score = bleu_score(candidate_corpus, references_corpus)
print(f'BLEU score on test data: {score*100}')
assert score*100 > 40, "The BLEU score is too low."

<div class="alert alert-block alert-info">
<b>Conclusion</b>
</div>

In this notebook:
* We learned how recurrent neural networks can be used to build a sequence-to-sequence model.
* We trained a sequence-to-sequence model for statistical machine translation.